In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
# 'hihello' 가르치기
# h가 들어갔을때 i가 나오도록
# X : 'hihell' Y : 'ihello'
idx2char= ['h','i','e','l','o']
x_data = [[0,1,0,2,3,3]]
y_data = [[1,0,2,3,3,4]]

x_one_hot= [[
                [1,0,0,0,0],
                [0,1,0,0,0],
                [1,0,1,0,0],
                [0,0,1,1,0],
                [1,0,0,1,0]
]]

In [8]:
np.array(x_one_hot).shape

(1, 5, 5)

In [4]:
# 사용변수

input_dim = 5 # input으로 들어가는 분류 차원 -> 자료 항목의 갯수(카테고리, 뉴메릭 상관없이)
num_classes = 5 # 몇개로 분류할 것이냐 => 예측할 항목 => Y 
batch_size = 1 # 전체 자료를 시퀀스별로 총 몇개로 나눌거냐
hidden_size = 5 # 내가 정해
sequence_length = 6 # 한 단위의 크기
learning_rate = 0.1

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])

# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size, state_is_tuple=True)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
# cell = tf.contrib.rnn.GRUCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state,
                                dtype=tf.float32)
X_for_fc = tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(
        inputs=X_for_fc, num_outputs = num_classes, activation_fn=None)# 레이어가 많을경우 활성함수 쓴다

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y,
                                        weights=weights)
#
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs,axis=2)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(4000):
    l,_ = sess.run([loss,train], feed_dict={X:x_one_hot, Y:y_data})
    result = sess.run(prediction, feed_dict={X:x_one_hot})
    if step % 100 == 0:
        print ("Step:",step,"Loss:",l,"Prediction:", result)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("Prediction:","".join(result_str))